In [1]:
# Dependencies import.
import pandas as pd
import numpy as np
import requests
import time
from citipy import citipy
from datetime import datetime

# Import API key.
from config import weather_api_key

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Create a random set of latititude/longitudes combos.
lats = np.random.uniform(low = -90.000, high = 90.000, size = 2000)
lngs = np.random.uniform(low = -180.000, high = 180.000, size = 2000)

lat_lngs = zip(lats, lngs)

# Create a list from the lat_lngs zip.
coordinates = list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []

# Loop through the coordinates and find the closest city for each one
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # Add the city to the list if it is unique.
    if city not in cities:
        cities.append(city)

# Print the length of the unique cities list
len(cities)

726

In [4]:
# Create an empty list to hold the weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through the cities in the list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Current Description": city_desc,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | hamilton
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | albany
Processing Record 5 of Set 1 | severo-kurilsk
Processing Record 6 of Set 1 | mrirt
City not found. Skipping...
Processing Record 7 of Set 1 | saleaula
City not found. Skipping...
Processing Record 8 of Set 1 | port elizabeth
Processing Record 9 of Set 1 | puerto ayora
Processing Record 10 of Set 1 | morehead
Processing Record 11 of Set 1 | busselton
Processing Record 12 of Set 1 | buncrana
Processing Record 13 of Set 1 | hobart
Processing Record 14 of Set 1 | kapaa
Processing Record 15 of Set 1 | chuy
Processing Record 16 of Set 1 | datong
Processing Record 17 of Set 1 | iqaluit
Processing Record 18 of Set 1 | angoche
Processing Record 19 of Set 1 | lagoa
Processing Record 20 of Set 1 | akonolinga
Processing Record 21 of Set 1 | guerrero negro
Processing Record 22 of Set 1 

In [5]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed
0,Ushuaia,AR,2022-07-08 02:20:08,-54.8000,-68.3000,few clouds,29.86,74,20,11.50
1,Hamilton,US,2022-07-08 02:13:32,39.1834,-84.5333,scattered clouds,81.88,72,40,5.75
2,Punta Arenas,CL,2022-07-08 02:20:08,-53.1500,-70.9167,clear sky,21.31,100,0,5.75
3,Albany,US,2022-07-08 02:15:15,42.6001,-73.9662,overcast clouds,73.81,77,100,2.59
4,Severo-Kurilsk,RU,2022-07-08 02:20:08,50.6789,156.1250,broken clouds,56.52,85,71,8.25
5,Port Elizabeth,ZA,2022-07-08 02:18:25,-33.9180,25.5701,clear sky,55.99,88,0,4.76
6,Puerto Ayora,EC,2022-07-08 02:20:09,-0.7393,-90.3518,scattered clouds,71.55,86,26,1.99
7,Morehead,US,2022-07-08 02:15:47,37.2711,-87.1764,clear sky,73.85,99,0,0.00
8,Busselton,AU,2022-07-08 02:20:09,-33.6500,115.3333,overcast clouds,61.30,81,87,12.39
9,Buncrana,IE,2022-07-08 02:20:09,55.1333,-7.4500,overcast clouds,57.22,88,100,6.87


In [6]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label = "City_ID")